### 네이버 검색 결과 크롤링 - 수원맛집

In [26]:
import requests
import pandas as pd
from urllib.parse import quote
from bs4 import BeautifulSoup

- requests.get()으로 데이터를 가져온 경우

In [27]:
url = 'https://search.naver.com/search.naver?where=view&sm=tab_jum&query=' + quote('수원맛집')
res = requests.get(url)
soup = BeautifulSoup(res.text)

In [28]:
lis = soup.select('.bx._svp_item')
len(lis)

30

- Selenium을 이용해서 데이터 가져오기

In [29]:
from selenium import webdriver
import warnings
warnings.filterwarnings('ignore')

driver = webdriver.Chrome('C:/Users/YONSAI/Downloads/chromedriver')
driver.get(url)

In [30]:
# //*[@id="main_pack"]/section/div/div[2]/panel-list/div[1]/more-contents/div/ul/li[1]

In [31]:
# View 페이지를 아래로 내려서 많은 데이터 확보
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
import time

body = driver.find_element(By.TAG_NAME, 'body')
for _ in range(30):
    body.send_keys(Keys.PAGE_DOWN)
    time.sleep(1)

In [32]:
soup = BeautifulSoup(driver.page_source)
lis = soup.select('.bx._svp_item')
len(lis)

110

In [33]:
driver.close()

- 날짜, 제목, 내용, 크롤링

In [34]:
li = lis[0]
date = li.select_one('.sub_time.sub_txt').get_text().strip()
date

'2023.04.05.'

In [35]:
title = li.select_one('.api_txt_lines.total_tit._cross_trigger').get_text().strip()
title

'일품이던 수원맛집베스트10'

In [36]:
content = li.select_one('.api_txt_lines.dsc_txt').get_text().strip()
content[:200]

'하러 수원맛집베스트10에 속한다는 갈비집을 다녀왔어요. 동네 주민들 뿐만 아니라 멀리서도 많은 사람들이 찾아가는 곳이라는데 소문대로 고퀄리티의 식사를 즐길 수 있던 곳이었어요. 고반가든 서수원점 주소 : 경기...'

- 날짜 처리

In [37]:
from datetime import datetime, timedelta

today = datetime.now()
today

datetime.datetime(2023, 4, 25, 12, 30, 38, 440577)

In [38]:
# 45분 전
date = today - timedelta(minutes=45)
date.strftime('%Y.%m.%d')

'2023.04.25'

In [39]:
# 23시간 전
date = today - timedelta(hours=23)
date.strftime('%Y.%m.%d')

'2023.04.24'

In [40]:
# 어제
date = today - timedelta(1)
date.strftime('%Y.%m.%d')

'2023.04.24'

In [41]:
# 2일 전
date = today - timedelta(2)
date.strftime('%Y.%m.%d')

'2023.04.23'

In [42]:
def conver_date(s):
    if len(s) == 11:            # ''2023.04.23.'
        return s[:-1]
    
    today = datetime.now()
    if '어제' in s:
        date = today - timedelta(1)
    elif '일' in s:
        date = today - timedelta(int(s[:-3]))
    elif '시간' in s:
        date = today - timedelta(int(s[:-4]))
    elif '분' in s:
        date = today - timedelta(int(s[:-3]))
    
    return date.strftime('%Y.%m.%d')

- 데이터프레임 만들기

In [43]:
lines = []
for li in lis:
    date = li.select_one('.sub_time.sub_txt').get_text().strip()
    date = conver_date(date)
    title = li.select_one('.api_txt_lines.total_tit._cross_trigger').get_text().strip()
    content = li.select_one('.api_txt_lines.dsc_txt').get_text().strip()
    lines.append({'날짜':date, '제목':title, '내용':content})

In [45]:
df = pd.DataFrame(lines)
df.tail()

,날짜,제목,내용
105,2023.04.14,수원 맛집 가성비와 분위기 좋은 미도인,서울에서 핫한 미도인의 맛을 수원에서도 느낄 수 있어서 좋았어요 ! 경기 수원시 팔...
106,2023.04.13,"수원 맛집 소담화로 소갈비살 추천, 주차 정보",고기도 진짜 맛도리에요 >_< 가족외식 장소로도 회식장소로도 추천드려욤 #수원소고기...
107,2023.04.02,수원 맛집 호매실 돈가스 맛집 이백장돈가스 수원호매실점,#이백장돈가스 #이백장돈가스수원호매실점 #호매실돈가스 맛집 #호매실돈까스맛집 #수원...
108,2023.04.23,(수원맛집) 영양 가득 보양식 백세삼계탕,곳으로 수원 나가는 초입인 이목동에 위치해 잇어 접근성이 참 좋기도 합니다 지인이 ...
109,2023.04.16,재방문 200% 최애닭강정 <아들닭강정> 수원 맛집 내돈내산 주차가능,다음에도 맷집이네 놀러오세요~ #수원닭강정맛집 #수원치킨맛집 #수원통닭맛집리스트 #...
